**TensorFlow chapter 2 :**<br>
**In this chapter we will cover three topic's.**
* Managing Graphs.
* Lifecycle of a Node value.
* Linear Regression with TensorFlow 

<center><h1> Managing Graphs</h1></center>

Any **Node** we create is automatically added to the **default graph**.

In [ ]:
import tensorflow as tf

'''creating a variable'''
x = tf.Variable(1 , name = 'x')
'''checking whether the variable is stored as node value in the default graph.'''
x.graph is tf.get_default_graph()

So we know that a variable is stored in a default graph as a node.  <br>
In most of the cases this is fine  , but sometimes we may **want to manage multiple independent graphs**.<br>
We can do this by creating a **new graph** and **temporarily** making it the **default graph** inside a **with** block , like so :

In [ ]:
'''creating a temporary graph'''
graph = tf.Graph()
with graph.as_default():
    '''Creating a variable in temporary graph.'''
    x1 = tf.Variable(2 , name = 'x1')
    
x1.graph is graph 

In [ ]:
x1.graph is tf.get_default_graph()

In jupyter it is common to run the same commands more than once while we are experimenting . As a result , we may end up with a default graph containing many containing many duplicate nodes. One solution is to restart the jupyter kernel , but a more convinent solution is to just reset the default graph by running **tf.reset_default_graph()**.

<center> <h1> Lifecycle of a <b>NODE VALUE</b> </h1> </center>

When we evaluate a node , **TensorFlow automatically determines the set of nodes that it depends on and it evaluates these nodes first.**<br>
For example :

In [ ]:
'''first lets reset the graph'''
tf.reset_default_graph()

'''variables'''
w = tf.constant(2)
x = w + 3
y = x + 5
z = x * 3

'''initializing a session'''
with tf.Session() as session:
    print(y.eval()) #10
    print(z.eval()) #15

* First , the code will define a very simple graph. (**Construction phase**)
* Then it starts the session 
* Then it runs the graph to evaluate **y**.  (**Execution phase**).
     * TensorFlow automatically detects that **y is dependent on x** and **x is dependent on w**.
     * First it will evaluate **w** , then **x**, then **y** .
     * And returns the value of **y**.
* Finally the code runs to the graph to evaluate **z**
     * TensorFlow detects that **z is dependent on x  and x is depented on w**
     * It will first evaluate **w** , then **x** , then **z** ,and returns the value of **z**.
* If we notice that TensorFlow does not reuses the result of previous evaluation **w** and **x**. Inshort , the preceding code evaluates **w** and **x** twice.

<br>
All **node values** are dropped between graph runs , except variable values , which are maintained by the session across the graph runs.<br>
A Variable starts its life when its initializer is run , and it ends when the session is closed.
<br>
If we want to evaluate **y and z** effeciently , without evaluating **w** and **x** twice as we did in the previous code , we must ask TensorFlow to evaluate both **y** and **x** in just one graph run , Example


In [ ]:
with tf.Session() as session :
    '''Commanding TensorFlow to evaluate y and z , without evaluating w and x twice'''
    y_val , z_val = session.run([y , z ])
    print(y_val)
    print(z_val)

<center><h1><b>Linear Regression</b> with TensorFlow</h1></center>

* **TensorFlow operations (ops)** can take any number of of inputs and produce any number of outputs . For example , the addition and multiplication  ops each take two inputs and produce one output. Constant and variables take no input , they are called as **source ops**
* The inputs and outputs are multidimensional arrays, called as **TENSORS** (hence the name tensor flow).
* Just like NumPy arrays , **tensors** have a type and a shape. In fact , the Python API tensors are simply represented by **NumPy ndarrays**. We know that numpy ndarrays typically contain floats , but we can also use them to carry **strings** . In the examples so far , the **tensors** just contained a single scalar value.
* Now we will write a code which manipulates **2D arrays** to perform **Linear Regression** on the **Boston House pricing dataset**.

In [ ]:
'''importing the Boston House pricing dataset'''
from sklearn.datasets import load_boston
housing = load_boston()

1. **Now lets add a bias input feature ($x_{0}$ =   1)  to all training instances.** <br>
note : - Formula of **Linear Regression : <h2> $h(x) = \theta_{0} + x_{1}*\theta_{1}  + ...... + x_{n} * \theta_{n} $ </h2>, where $\theta_{0}$ is the bias value or intercept , $x_{1}...x{n}$ are the independent variables , and $\theta_{1}...\theta_{n}$ are the feature parameters or slope . Formula in matrix format is $\theta^T\cdotp x$** <br> 


In [ ]:
import numpy as np

m , n = housing.data.shape
'''adding or concating bias input feature'''
housing_plus_bias = np.c_[np.ones( (m , 1) ) , housing.data]

Creating two **TensorFlow constant nodes , X and y**<br>
* **X** =  independent feature matrix.
* **y** = dependent feature vectore.

In [ ]:
X = tf.constant(housing_plus_bias , dtype = tf.float32 , name = "X")
y = tf.constant(housing.target.reshape(-1 , 1) , dtype = tf.float32 , name = "y")

'''even create X transpose which will be usefull for theta evaluation.'''
XT = tf.transpose(X)

* Now that we have **X** and **y** , it is time to fit the parameter  $\theta$ .<br>
* Here we will use **Normal Equation** to find the optimized value of $\theta$.
* Formula for **Normal Equation**  : <h2>$\theta = (X^T \cdotp X)^{-1} \cdotp X^T \cdotp y$ </h2>
* Here we will use **Matrix Operation** provide by TensorFlow eg , matmul() , matrix_inverse() . 

In [ ]:
'''This line of code does not performs any kin of computaion , instead it creates a nodes
in the graph'''
theta = tf.matmul( tf.matmul( tf.matrix_inverse( tf.matmul(XT , X ) ) , XT ) , y)

'''Initializing session'''
with tf.Session() as session:
    '''evaluating theta'''
    theta_value = theta.eval()
print(theta_value)

**Below are the parameters ($\theta$) learned by our model.**<br>
* The main benefit of this **code versus computing the Normal Equation** directly using **NumPy** is that TensorFlow will automatically run this on your **GPU** card if you have one.
<br>
<br>
* **In the next chapter we will implement Gradient Descent to fit the parameter $\theta$  manually and by using an Optimizer (tf.train.Gradient).**